In [3]:
import sys
!{sys.executable} -m pip install qcodes

     |████████████████████████████████| 1.1 MB 1.3 MB/s eta 0:00:01
  Using cached PyVISA-1.11.3-py3-none-any.whl (189 kB)
  Using cached requirements-parser-0.2.0.tar.gz (6.3 kB)
     |████████████████████████████████| 3.0 MB 2.5 MB/s eta 0:00:01
     |████████████████████████████████| 101 kB 2.6 MB/s ta 0:00:01
     |████████████████████████████████| 4.0 MB 177 kB/s eta 0:00:01
     |████████████████████████████████| 759 kB 1.2 MB/s eta 0:00:01
     |████████████████████████████████| 58 kB 817 kB/s eta 0:00:01
     |████████████████████████████████| 127 kB 1.7 MB/s eta 0:00:01
     |████████████████████████████████| 16.1 MB 53 kB/s eta 0:00:011


     |████████████████████████████████| 297 kB 1.3 MB/s eta 0:00:01
  Using cached opencensus_context-0.1.2-py2.py3-none-any.whl (4.4 kB)
     |████████████████████████████████| 93 kB 1.8 MB/s eta 0:00:01
     |████████████████████████████████| 198 kB 2.1 MB/s eta 0:00:01
     |████████████████████████████████| 142 kB 1.6 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 513 kB/s eta 0:00:01
  Using cached pyasn1_modules-0.2.8-py2.py3-none-any.whl (155 kB)
     |████████████████████████████████| 236 kB 2.4 MB/s eta 0:00:01
  Using cached pyasn1-0.4.8-py2.py3-none-any.whl (77 kB)
     |████████████████████████████████| 156 kB 2.5 MB/s eta 0:00:01


  Created wheel for requirements-parser: filename=requirements_parser-0.2.0-py3-none-any.whl size=7655 sha256=23bbb84dc756c345ea88f57b0366f1790f8774acfd2c6a3b52a3187b3eaeec5f
  Stored in directory: /Users/fabrizioberr/Library/Caches/pip/wheels/79/d2/70/93e0b1c93d75d9e2fe4b996772f7f760186b499f9ed07b572b
  Created wheel for websockets: filename=websockets-8.1-cp39-cp39-macosx_10_9_x86_64.whl size=65246 sha256=b3b72ffee036b96276c5e27dea58762d4179febf3e732e65f639dc536163f3a8
  Stored in directory: /Users/fabrizioberr/Library/Caches/pip/wheels/d8/b9/a0/b97b211aeda2ebd6ac2e43fc300d308dbf1f9df520ed390cae
  Created wheel for wrapt: filename=wrapt-1.12.1-cp39-cp39-macosx_10_9_x86_64.whl size=30854 sha256=e5a2e12a99721027e7d29e1a00a22882f157337618b229e6687b9640285b04d4
  Stored in directory: /Users/fabrizioberr/Library/Caches/pip/wheels/98/23/68/efe259aaca055e93b08e74fbe512819c69a2155c11ba3c0f10
Successfully built requirements-parser websockets wrapt
  Attempting uninstall: numpy
    Found exist

In [1]:
import panel as pn
import numpy as np
import time
import holoviews as hv
from panel.widgets import Button, Toggle
from panel import Row, Column
from qcodes.utils.dataset.doNd import do0d
from holoviews import opts
from holoviews.streams import Pipe
from holoviews import opts
from tornado.ioloop import PeriodicCallback
from tornado import gen
from qcodes.utils.dataset.doNd import do0d
hv.extension('bokeh')

ModuleNotFoundError: No module named 'qcodes'

In [5]:
class LiveStream():
    """
    Class for live streaming data, without saving and with the possibility to add sliders/controllers

        Attributes
        ----------
        data_func: Function that returns the data which should be livestreamed
                   ex. alazar_channel.get
        sliders: Dict of the form {'name': (func_set, min_value,max_value)

        Methods
        -------
        measure
            do0d on data_func

    """

    def __init__(self,data_func,sliders):
        self.data_func = data_func
        self.pipe = Pipe(data=[])
        self.image_dmap = hv.DynamicMap(hv.Image, streams=[self.pipe])
        self.image_dmap.opts(cmap = 'Magma', xlim=(-0.6, 0.6), ylim=(-0.6, 0.6))
        self.measure_button = Button(name='measure',button_type = 'primary',
                              width=100)
        self.run_id_in_text = 'None'
        self.run_id_widget = pn.widgets.TextInput(name='run_id', value=self.run_id_in_text)


        def measure(event):
            data_do0d = do0d(self.data_func)
            self.run_id_widget.value = f'{data_do0d[0].run_id}'

        self.measure_button.on_click(measure)
        self.plot_id_in_text = 'test'
        
        self.sliders = []
        self.sliders_func = []
        for key in sliders.keys():
            self.sliders_func.append(sliders[key][0])
            self.sliders.append(pn.widgets.FloatSlider(name=str(key),
                                              start=sliders[key][1],
                                              end=sliders[key][2],
                                              step=sliders[key][3],
                                              value=sliders[key][4]))
        self.dis()


    def dis(self):
        col = (self.measure_button,
                self.run_id_widget,) + tuple(self.sliders)
        refresh_period = 500
        port = 12359
        self.video_mode_callback = PeriodicCallback(self.data_grabber, refresh_period)


        self.video_mode_server = Row(self.image_dmap, Column(*col)).show(port=port,threaded=True)
        self.video_mode_callback.start() 

    @gen.coroutine  
    def data_grabber(self):
        for i, func in enumerate(self.sliders_func):
            func(self.sliders[i].value)
        self.pipe.send(self.data_func())